# itch_wang_res_comparison

#### Juan Camilo Henao Londono - 07.03.2019
#### AG Guhr - Universitaet Duisburg-Essen

## Step 1

Obtain the data in the same format of S. Wang for the stock AAPL 2008.01.07. It is used the algorithm I implemented.

In [1]:
import gzip
import numpy as np
import pandas as pd

import itch_taq_trade_sign_classification_test_basic as itch_sign_clas

In [2]:
ticker = 'AAPL'
year = '2008'
month = '01'
day = '07'

In [3]:
times_signs, trade_signs, vol_signs, price_signs = itch_sign_clas.itch_taq_trade_signs_load_test(ticker, year,
                                                                                                 month, day)
id_trades = itch_sign_clas.itch_taq_trade_signs_eq1_ms_test(ticker, trade_signs, price_signs, year, month, day)
teo_s, exp_eq2_s, times_s = itch_sign_clas.itch_taq_trade_signs_eq2_s_test(ticker, times_signs, trade_signs,
                                                                           id_trades, year, month, day)
_, exp_eq3_s, _ = itch_sign_clas.itch_taq_trade_signs_eq3_s_test(ticker, times_signs, trade_signs,
                                                                 vol_signs, id_trades, year, month, day)

ITCH data
itch_taq_trade_signs_load_test
Processing data for the stock AAPL the 2008.01.07
Time step: -ms
Accuracy of the trade sign classification for consecutive trades in ms for the stock AAPL the 2008.01.07
For consecutive trades in ms:
Accuracy of the classification: 83.03 %
Number of identified trades: 120287
Number of matches: 99871
Accuracy of the trade sign classification for number imbalance of  trades in seconds for the stock AAPL the 2008.01.07
Reducing the trades to 1 per second:
Accuracy of the classification: 82.38 %
Number of identified trades: 22200
Number of matches: 18289
Accuracy of the trade sign classification for volume imbalance of  trades in ms for the stock AAPL the 2008.01.07
Reducing the trades to 1 per millisecond:
Accuracy of the classification: 80.3 %
Number of identified trades: 22200
Number of matches: 17827


In [4]:
d1 = {'Time': times_signs, 'Teo': trade_signs, 'Exp': id_trades, 'Price': price_signs}
transactions_juan = pd.DataFrame(data=d1).astype(int)
transactions_juan['Price'] = transactions_juan['Price'] / 10000

In [5]:
d2 = {'Time': times_s * 1000, 'Teo': teo_s, 'Eq2': exp_eq2_s, 'Eq3': exp_eq3_s}
persecond_juan = pd.DataFrame(data=d2).astype(int)

## Step 2

Load S. Wang data.

In [6]:
transactions_wang = pd.read_csv('SignCompare_transactions_20080107_AAPL.txt', sep='   ',
                                usecols=(1,3,4,5), header=None, engine='python')
transactions_wang.columns = ['Time', 'Teo', 'Exp', 'Price']

In [7]:
persecond_wang = pd.read_csv('SignCompare_persecond_20080107_AAPL.txt', sep='   ',
                             usecols=(0,2,3,4), header=None, engine='python')
persecond_wang.columns = ['Time', 'Teo', 'Eq2', 'Eq3']

## Step 3

Compare the data to see where are the difference

In [8]:
trans_time_comp = np.sum(transactions_wang['Time'] == transactions_juan['Time']) / len(transactions_wang['Time'])
perse_time_comp = np.sum(persecond_wang['Time'] == persecond_juan['Time']) / len(persecond_wang['Time'])

print('The similarity of the time in the transaction time used is ', trans_time_comp * 100, '%')
print('The similarity of the time in the presecond time used is ', perse_time_comp * 100, '%')

The similarity of the time in the transaction time used is  100.0 %
The similarity of the time in the presecond time used is  100.0 %


In [9]:
trans_teo_comp = np.sum(transactions_wang['Teo'] == transactions_juan['Teo']) / len(transactions_wang['Teo'])
perse_teo_comp = np.sum(persecond_wang['Teo'] == persecond_juan['Teo']) / len(persecond_wang['Teo'])

print('The similarity of the reference trade signs values for the transactions is ', trans_teo_comp * 100, '%')
print('The similarity of the reference trade signs values for the persecond is ', perse_teo_comp * 100, '%')

The similarity of the reference trade signs values for the transactions is  100.0 %
The similarity of the reference trade signs values for the persecond is  100.0 %


In [10]:
trans_price_comp = np.sum(transactions_wang['Price'] == transactions_juan['Price']) / len(transactions_wang['Price'])

print('The similarity of the prices for the transactions is ', trans_price_comp * 100, '%')

The similarity of the prices for the transactions is  100.0 %


In [11]:
trans_exp_comp = np.sum(transactions_wang['Exp'] == transactions_juan['Exp']) / len(transactions_wang['Exp'])
perse_eq2_comp = np.sum(persecond_wang['Eq2'] == persecond_juan['Eq2']) / len(persecond_wang['Eq2'])
perse_eq3_comp = np.sum(persecond_wang['Eq3'] == persecond_juan['Eq3']) / len(persecond_wang['Eq3'])

print('The similarity of the experimental result for the transactions is ', trans_exp_comp * 100, '%')
print('The similarity of the Eq. 2 result for the presecond is ', perse_eq2_comp * 100, '%')
print('The similarity of the Eq. 3 result for the presecond is ', perse_eq3_comp * 100, '%')

The similarity of the experimental result for the transactions is  94.39673447670987 %
The similarity of the Eq. 2 result for the presecond is  79.77927927927928 %
The similarity of the Eq. 3 result for the presecond is  87.6891891891892 %


## Step 4

Check the first difference in every experimental result and determine the error.

In [12]:
trans_exp_diff = np.where(transactions_wang['Exp'] != transactions_juan['Exp'])[0]

print('The total of different values between the experimental results are ', len(trans_exp_diff))
print('The first four different values are located in the positions ', trans_exp_diff[:10])

The total of different values between the experimental results are  6740
The first four different values are located in the positions  [  0  27  86  87  88  95 105 107 151 152]


In [13]:
print('The first value of Wang is {} and of Juan is {}'
      .format(transactions_wang['Exp'][0], transactions_juan['Exp'][0]))

The first value of Wang is -1 and of Juan is 1


The reason of this error is due to the fact I just use the values from the open market time. Mi first value is calculated using the difference between the first and last value `diff = price_signs[0] - price_signs[-1]`, then my first value is error prone.

In [14]:
wang = 0
juan = 0

for val in trans_exp_diff[1:]:
    
    print('The value in pos. {} of Wang is {} and of Juan is {}'
          .format(val, transactions_wang['Exp'][val], transactions_juan['Exp'][val]))
    print('To check which value is correct, we need to find the value in the position {} and {}'
          .format(val, val - 1))
    sign = np.sign(price_signs[val] - price_signs[val - 1])
    print('The price in pos. {} is {} and in position {} is {}. Then the trade sign must be {}'
              .format(val, price_signs[val], val - 1, price_signs[val - 1], sign))
    if (sign == 0):
        print(' '.join(('As the sign is zero, we use the trade sign value in pos. {}. For Wang that value'
              + ' is {} and for Juan that value is {}. The real value of that position is {}').split())
              .format(val - 1, transactions_wang['Exp'][val - 1], transactions_juan['Exp'][val - 1],
                      trade_signs[val - 1]))
        if (transactions_wang['Exp'][val - 1] == trade_signs[val - 1]
            and transactions_juan['Exp'][val - 1] == trade_signs[val - 1]):
            print('Both were right')
            wang += 1
            juan += 1
        elif (transactions_wang['Exp'][val - 1] == trade_signs[val - 1]):
            print('Wang was right')
            wang += 1
        else:
            print('Juan was right')
            juan += 1
    else:
        print('The real value of that position is {}'.format(trade_signs[val]))
        if (trade_signs[val] == transactions_wang['Exp'][val]):
            print('Wang was right')
            wang += 1
        else:
            print('Juan was right')
            juan += 1
    print()

print('Wang was right {} times and Juan was right {} times'.format(wang, juan))    
print()

The value in pos. 27 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 27 and 26
The price in pos. 27 is 1807900.0 and in position 26 is 1808000.0. Then the trade sign must be -1.0
The real value of that position is 1.0
Wang was right

The value in pos. 86 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the position 86 and 85
The price in pos. 86 is 1807800.0 and in position 85 is 1807800.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 85. For Wang that value is 1 and for Juan that value is 1. The real value of that position is 1.0
Both were right

The value in pos. 87 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the position 87 and 86
The price in pos. 87 is 1807800.0 and in position 86 is 1807800.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 86. For

Wang was right

The value in pos. 882 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 882 and 881
The price in pos. 882 is 1808900.0 and in position 881 is 1808900.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 881. For Wang that value is -1 and for Juan that value is -1. The real value of that position is -1.0
Both were right

The value in pos. 883 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 883 and 882
The price in pos. 883 is 1808900.0 and in position 882 is 1808900.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 882. For Wang that value is 1 and for Juan that value is -1. The real value of that position is 1.0
Wang was right

The value in pos. 884 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 884 and 883
The price in 

To check which value is correct, we need to find the value in the position 3596 and 3595
The price in pos. 3596 is 1821500.0 and in position 3595 is 1821400.0. Then the trade sign must be 1.0
The real value of that position is -1.0
Wang was right

The value in pos. 3600 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the position 3600 and 3599
The price in pos. 3600 is 1821600.0 and in position 3599 is 1821400.0. Then the trade sign must be 1.0
The real value of that position is -1.0
Wang was right

The value in pos. 3601 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the position 3601 and 3600
The price in pos. 3601 is 1821600.0 and in position 3600 is 1821600.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 3600. For Wang that value is -1 and for Juan that value is 1. The real value of that position is -1.0
Wang was right

The value in pos. 3602 of Wan

The value in pos. 5579 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 5579 and 5578
The price in pos. 5579 is 1812600.0 and in position 5578 is 1812600.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 5578. For Wang that value is 1 and for Juan that value is -1. The real value of that position is 1.0
Wang was right

The value in pos. 5586 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the position 5586 and 5585
The price in pos. 5586 is 1812400.0 and in position 5585 is 1812100.0. Then the trade sign must be 1.0
The real value of that position is -1.0
Wang was right

The value in pos. 5587 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the position 5587 and 5586
The price in pos. 5587 is 1812400.0 and in position 5586 is 1812400.0. Then the trade sign must be 0.0
As the sign is zero, we use the 

Juan was right

The value in pos. 7708 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 7708 and 7707
The price in pos. 7708 is 1828600.0 and in position 7707 is 1828600.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 7707. For Wang that value is 1 and for Juan that value is -1. The real value of that position is 1.0
Wang was right

The value in pos. 7731 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 7731 and 7730
The price in pos. 7731 is 1828700.0 and in position 7730 is 1828700.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 7730. For Wang that value is -1 and for Juan that value is -1. The real value of that position is 1.0
Juan was right

The value in pos. 7797 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 7797 and 7796


...

The value in pos. 107265 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 107265 and 107264
The price in pos. 107265 is 1754600.0 and in position 107264 is 1754600.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 107264. For Wang that value is 1 and for Juan that value is -1. The real value of that position is 1.0
Wang was right

The value in pos. 107333 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 107333 and 107332
The price in pos. 107333 is 1753200.0 and in position 107332 is 1753400.0. Then the trade sign must be -1.0
The real value of that position is 1.0
Wang was right

The value in pos. 107334 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 107334 and 107333
The price in pos. 107334 is 1753200.0 and in position 107333 is 1753200.0. Then the trade sign must be 0.0


Wang was right

The value in pos. 110312 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 110312 and 110311
The price in pos. 110312 is 1758800.0 and in position 110311 is 1758800.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 110311. For Wang that value is -1 and for Juan that value is -1. The real value of that position is -1.0
Both were right

The value in pos. 110313 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 110313 and 110312
The price in pos. 110313 is 1758800.0 and in position 110312 is 1758800.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 110312. For Wang that value is 1 and for Juan that value is -1. The real value of that position is 1.0
Wang was right

The value in pos. 110314 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in

The price in pos. 113883 is 1733700.0 and in position 113882 is 1733700.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 113882. For Wang that value is 1 and for Juan that value is -1. The real value of that position is 1.0
Wang was right

The value in pos. 113951 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the position 113951 and 113950
The price in pos. 113951 is 1733200.0 and in position 113950 is 1732800.0. Then the trade sign must be 1.0
The real value of that position is -1.0
Wang was right

The value in pos. 113952 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the position 113952 and 113951
The price in pos. 113952 is 1733300.0 and in position 113951 is 1733200.0. Then the trade sign must be 1.0
The real value of that position is 1.0
Juan was right

The value in pos. 113953 of Wang is -1 and of Juan is 1
To check which value is correct, we ne

To check which value is correct, we need to find the value in the position 116571 and 116570
The price in pos. 116571 is 1730900.0 and in position 116570 is 1730900.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 116570. For Wang that value is 1 and for Juan that value is -1. The real value of that position is 1.0
Wang was right

The value in pos. 116598 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 116598 and 116597
The price in pos. 116598 is 1731000.0 and in position 116597 is 1731200.0. Then the trade sign must be -1.0
The real value of that position is 1.0
Wang was right

The value in pos. 116599 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 116599 and 116598
The price in pos. 116599 is 1731000.0 and in position 116598 is 1731000.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos.


The value in pos. 119532 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 119532 and 119531
The price in pos. 119532 is 1765900.0 and in position 119531 is 1766300.0. Then the trade sign must be -1.0
The real value of that position is 1.0
Wang was right

The value in pos. 119533 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 119533 and 119532
The price in pos. 119533 is 1765900.0 and in position 119532 is 1765900.0. Then the trade sign must be 0.0
As the sign is zero, we use the trade sign value in pos. 119532. For Wang that value is 1 and for Juan that value is -1. The real value of that position is 1.0
Wang was right

The value in pos. 119534 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the position 119534 and 119533
The price in pos. 119534 is 1765900.0 and in position 119533 is 1765900.0. Then the trade sign must be 0.0

## Step 5

As there is no reason for a difference in the results as the theoretical values are all the same, I will reproduce Eq. 1, Eq. 2 and Eq. 3 from the S. Wang file.

In [15]:
identified_trades = np.zeros(len(transactions_wang['Price']))
price = np.array(transactions_wang['Price'])

for t_idx, t_val in enumerate(identified_trades):

    diff = price[t_idx] - price[t_idx - 1]

    if (diff):

        identified_trades[t_idx] = np.sign(diff)

    else:

        identified_trades[t_idx] = identified_trades[t_idx - 1]

assert not np.sum(identified_trades == 0)

In [16]:
trans_res_comp = np.sum(identified_trades == transactions_juan['Exp']) / len(identified_trades)
print('The similarity of the results for the transactions is ', trans_res_comp * 100, '%')

The similarity of the results for the transactions is  100.0 %
